![](https://img-d02.moneycontrol.co.in/images/common/header/logo.png?v=0.1?raw=true)

# Moneycontrol scraping
### First we need to scrape all the companies under Indian Market

In [17]:
from IPython.display import display, HTML
from multiprocessing.dummy import Pool
from bs4 import BeautifulSoup
import pandas as pd
import requests

#### The function which scraps links like:

https://www.moneycontrol.com/india/stockpricequote/A  
https://www.moneycontrol.com/india/stockpricequote/B  
.
.
.  
https://www.moneycontrol.com/india/stockpricequote/Z  
https://www.moneycontrol.com/india/stockpricequote/others

In [18]:
company_list=[]
def get_company_list(url):    
    page_response = requests.get(url, timeout=5)
    soup = BeautifulSoup(page_response.content,"lxml")
    companies = soup.findAll("a", attrs={"class":"bl_12"})
    for company in companies:
        if "moneycontrol" in company["href"]:
            company_list.append([company.text,company["href"]])

#### This generates all links and sends to the function through multithreading

In [19]:
urls=[]
incomplete_url="https://www.moneycontrol.com/india/stockpricequote/"
for i in range (ord("A"),ord("Z")+1):
    urls.append(incomplete_url+chr(i))
urls.append((incomplete_url+"others"))
with Pool() as pool:
    (pool.map(get_company_list, urls))
df =  pd.DataFrame(company_list)
df.columns = ['Company Names', 'Company Links']
df.to_csv('company_list.csv', sep=',', header=None, index=None)

#### This is the table

In [20]:
print ("Dataframe 1:")
display(HTML(df.to_html()))

Dataframe 1:


,Company Names,Company Links
0,E I Granites,http://www.moneycontrol.com/india/stockpricequ...
1,E-Land Apparel,http://www.moneycontrol.com/india/stockpricequ...
2,E-Metals India,http://www.moneycontrol.com/india/stockpricequ...
3,E-Serve Intl,http://www.moneycontrol.com/india/stockpricequ...
4,E.Star Info,http://www.moneycontrol.com/india/stockpricequ...
5,E2E Networks,http://www.moneycontrol.com/india/stockpricequ...
6,Earnest Health,http://www.moneycontrol.com/india/stockpricequ...
7,East Buildtech,http://www.moneycontrol.com/india/stockpricequ...
8,East Coast Stee,http://www.moneycontrol.com/india/stockpricequ...
9,East India Sec,http://www.moneycontrol.com/india/stockpricequ...


#### Now get a perticular company and get the link to that Stock Website

In [34]:
company_name_given_by_user = "igarashi motors"
company_link=df[df["Company Names"].str.contains(company_name_given_by_user, case=False)]["Company Links"].iloc[0]
print("The link: "+company_link)

The link: http://www.moneycontrol.com/india/stockpricequote/electricequipment/igarashimotors/IM01
